In [5]:
import pandas as pd
import itertools
import numpy as np
import random
from collections import defaultdict
from torch.utils.data import Sampler
from pathlib import Path
from monai.data import CacheDataset
from pytorch_lightning import LightningDataModule
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    CenterSpatialCropd,
    ToTensord,
)


class DataModule(LightningDataModule):
    def __init__(
        self,
        path_to_center_folders,
        prediction_target_file_path,
        target,
        test_center,
        dim,
        max_batch_size=32,
    ):
        super().__init__()
        self.root = Path(path_to_center_folders)
        self.target = pd.read_csv(prediction_target_file_path).set_index("lesion")[
            target
        ]
        self.test_center = test_center
        self.max_batch_size = max_batch_size
        self.dim = dim

        self.train_transform = self.get_transform(augmented=True)
        self.val_transform = self.get_transform(augmented=True)
        self.test_transform = self.get_transform(augmented=False)
        
        self.centers = [c.name for c in self.root.iterdir()]

    def setup(self, *args, **kwargs):
        dev_centers = [c for c in self.centers if not c == self.test_center]

        # development data
        dev_data = list(
            itertools.chain(
                *[self.data_dir_to_dict(self.root / c) for c in dev_centers]
            )
        )
        train_data, val_data = self.grouped_train_val_split(dev_data, val_fraction=0.25)
        self.train_dataset = CacheDataset(train_data, self.train_transform)
        self.val_dataset = CacheDataset(val_data, self.val_transform)

        # test data
        test_data = self.data_dir_to_dict(self.root / self.test_center)
        self.test_dataset = CacheDataset(test_data, self.test_transform)

    def grouped_train_val_split(self, dev_data, val_fraction):
        all_patients = [x["patient"] for x in dev_data]
        random.shuffle(all_patients)

        split_ix = int(len(all_patients) * (1 - val_fraction))

        return dev_data[:split_ix], dev_data[split_ix:]

    def data_dir_to_dict(self, dir):
        return [
            {
                "img": str(lesion_path),
                "label": self.target.loc[lesion_path.name],
                "patient": lesion_path.name.split(".")[0][:-2],
            }
            for lesion_path in dir.iterdir()
            if lesion_path.name in self.target.index
        ]

    def train_dataloader(self):
        return get_dataloader(self.train_dataset, shuffle=True)

    def val_dataloader(self):
        return get_dataloader(self.val_dataset)

    def test_dataloader(self):
        return get_dataloader(self.test_dataset)

    def get_dataloader(self, dataset, shuffle=False):
        return DataLoader(
            dataset,
            batch_sampler=GroupedSampler(
                groups=[x["patient"] for x in self.test_dataset],
                shuffle=shuffle,
                max_batch_size=self.max_batch_size,
            ),
        )

    def get_transform(self, augmented=False):
        load = Compose([
                LoadImaged(keys=["img"]),
                EnsureChannelFirstd(keys=["img"]),
        ])

        if self.dim == 3:
            augmentation = Compose([
                RandFlipd(keys=['img'], prob=0.5, spatial_axis=0),
                RandFlipd(keys=['img'], prob=0.5, spatial_axis=1),
                RandFlipd(keys=['img'], prob=0.5, spatial_axis=2),
                RandomTranspose()
            ])
        elif self.dim == 2:
            augmentation = Compose([
                RandFlipd(keys=['img'], prob=0.5, spatial_axis=0),
                RandRotate90d(keys=['img'], prob=1, k=4)
            ])

        if augmented:
            return Compose([
                load,
                augmentation,
                ToTensord(keys=['img'])
            ])
        else: 
            return Compose([
                load,
                ToTensord(keys=['img'])
            ])


class GroupedSampler(Sampler):
    def __init__(self, groups, max_batch_size, shuffle=False):
        self.max_batch_size = max_batch_size
        self.shuffle = shuffle

        self.unique_groups = np.unique(groups)

        self.indices_per_group = defaultdict(list)
        self.count_per_group = defaultdict(int)

        for ix, group in enumerate(groups):
            self.indices_per_group[group].append(ix)
            self.count_per_group[group] += 1

        self.make_batches_from_groups()

    def make_batches_from_groups(self):
        self.all_batches = []
        current_batch = []
        current_batch_length = 0
        for group in self.unique_groups:
            group_length = self.count_per_group[group]
            if current_batch_length + group_length > self.max_batch_size:
                self.all_batches.append(current_batch)
                current_batch = self.indices_per_group[group].copy()
                current_batch_length = group_length
            else:
                current_batch.extend(self.indices_per_group[group])
                current_batch_length += group_length
        if current_batch:
            self.all_batches.append(current_batch)

    def __iter__(self):
        if self.shuffle:
            random.shuffle(self.all_batches)

        for batch in self.all_batches:
            yield batch

    def __len__(self):
        return len(self.all_batches)


In [4]:


list(permutations([0,1,2]))

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [6]:
transform = Compose([
        LoadImaged(keys=["img"]),
        EnsureChannelFirstd(keys=["img"]),
])

output = transform({'img':r'C:\Users\user\data\dl_radiomics\dim-3_size-128_method-crop_roi_size-10\amphia\PREM_AM_001_0.nii.gz'})

In [39]:
from monai.transforms import RandFlipd, Transform

class RandomTranspose(Transform):
    def __call__(self, data):
        axis_orderings = list(permutations([0,1,2]))
        chosen_ordering = random.sample(axis_orderings, 1)[0]
        ordering_including_batch = [0] + [ax+1 for ax in chosen_ordering]

        data['img'] = data['img'].permute(ordering_including_batch)

        return data






In [40]:
augmentation(output)



{'img': tensor([[[[0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           [0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           [0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           ...,
           [0.2570, 0.2570, 0.2570,  ..., 0.2575, 0.2575, 0.2575],
           [0.2570, 0.2570, 0.2570,  ..., 0.2575, 0.2575, 0.2575],
           [0.2570, 0.2570, 0.2570,  ..., 0.2575, 0.2575, 0.2575]],
 
          [[0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           [0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           [0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           ...,
           [0.2570, 0.2570, 0.2570,  ..., 0.2575, 0.2575, 0.2575],
           [0.2570, 0.2570, 0.2570,  ..., 0.2575, 0.2575, 0.2575],
           [0.2570, 0.2570, 0.2570,  ..., 0.2575, 0.2575, 0.2575]],
 
          [[0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           [0.2214, 0.2214, 0.2214,  ..., 0.2594, 0.2594, 0.2594],
           [0.221

[0, 3, 1, 2]